<p><img src="images/1line.png" width="100%" /></p>

<ul>
<li>The example below utilizes Businesses formation information from a Colorado site for Colorado Business entities.</li>

<li>Data Site: <a href="https://data.colorado.gov/resource/4ykn-tg5h.json">https://data.colorado.gov/resource/4ykn-tg5h.json</a>&nbsp;</li>
<li>Data JSON Format (one entity):</li>
</ul>
<pre>{	"entityid": "18861217679",
	"entityname": "DENVER UNION CORPROATION, Dissolved January 17, 1983",
	"principaladdress1": "1512 LARIMER STREET #760",
	"principalcity": "Denver",
	"principalstate": "CO",
	"principalzipcode": "80202",
	"entitystatus": "Voluntarily Dissolved",
	"jurisdictonofformation": "CO",
	"entitytype": "Corporation",
	"agentfirstname": "JOHN",
	"agentmiddlename": "F.",
	"agentlastname": "O'DEA",
	"agentprincipaladdress1": "1512 LARIMER STREET #760",
	"agentprincipalcity": "DENVER",
	"agentprincipalstate": "CO",
	"agentprincipalzipcode": "80202",
	"agentprincipalcountry": "US",
	"entityformdate": "1886-03-29T00:00:00.000"  }</pre>
<ul>
<li>Records are read from a file <span class="instructure_file_holder link_holder"><a class="instructure_file_link" title="bizdata_json.txt" href="data/bizdata_json.txt" >bizdata_json.txt</a> </span>containing data for "80211".</li>
<li>It illustrates how tor write multiple records from an external data source to a MongoDB database.</li>
<li><span>It also illustrates how to read data back out of the MongoDB database, loop though the data and display it as formatted text</span><span>.</span></li>
</ul>

In [1]:
# -*- coding: utf-8 -*-
"""
Manipulating COBiz MongoDB database
Created on Fri Nov 13 12:07:50 2020

@author: dawng
"""
import pymongo
import json
import datetime

import os   # allows us to connect to operating system
uri = os.environ.get("atlas-uri")
client = pymongo.MongoClient(uri)

dblist = client.list_database_names()
print(dblist)

# Create/Open a database to hold the raw JSON Colorado Business data
db = client['cobizdb']
cobiz = db.cobiz

# Check wheter db not in MongoDB
if 'cobizdb'not in dblist :
    # Read JSON data from a file
    fdata = open('bizdata_json.txt', "r")
   
    # get all of the text from the document
    text = fdata.read()

    # Load the JSON text from the URL into a dictionary using the json library
    js = json.loads(text)

    # Use insert_many to insert ALL of the CO Businesses
    new_biz = cobiz.insert_many(js)
    
    print(js[-1]["entityformdate"])   

# Find data for specific Businesses And Display it
# find all Businesses that were formaed as a "Corporation"
filters = {'entitytype': { '$eq': "Limited Liability Company" }}

for biz in cobiz.find(filters):
    
    # Use isoformat to convert to datetime THEN strftime to format output
    dt = datetime.datetime.fromisoformat(biz["entityformdate"])
    dstring = dt.strftime("%m-%d-%Y")
      
    # Use format to format output in columns
    print("{: <20} {: >12}  {: <20}".format(biz['entityname'], dstring, biz["entitystatus"]))

['bikesdb', 'cobizdb', 'earthquakesdb', 'sample_geospatial', 'sample_mflix', 'sample_weatherdata', 'admin', 'local']
MyDystopia LTD         11-20-2020  Good Standing       
know + heal, LLC       11-20-2020  Good Standing       
Alpine Imports, LLC    11-20-2020  Good Standing       
